In [14]:
import os
import sys
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus ,quote
from xml.etree import ElementTree

from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import array_contains, udf

from datetime import datetime, timedelta
from pymongo import MongoClient
import pandas as pd
import json
import re

## Spark Session 연결

In [15]:
## processing_load.py
spark = SparkSession \
    .builder \
    .appName("ojo_test") \
    .config("spark.mongodb.input.uri", "mongodb://localhost:27017") \
    .config("spark.mongodb.input.database","OJO") \
    .config("spark.mongodb.input.collection", "naver_web_api") \
    .config("packages org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()
sc =spark.sparkContext

# pymongo connect
client = MongoClient('localhost',27017) # mongodb 27017 port
db = client.OJO

## 네이버 웹문서 검색 API

In [23]:
# naver application clientID,secret
client_id = "6E6mBjvxbdEgVk6Prqgd"
client_secret = "0ZjWnYiBjr"


# quote가 인코딩을 변경(UTF8로 url에 넘겨주기 위해서)
# 내가 입력한 값을 기준으로 parse
query = quote(input("검색 질의: "))
idx = 0 # 요청 개수
display = 100 # 100개 단위로 가져온다
start = 1
end = 100

# 데이터 프레임 생성
web_df = pd.DataFrame(columns=("Title", "Link", "Description"))

for start_index in range(start, end, display):
    # json 결과
    url = "https://openapi.naver.com/v1/search/webkr?query=" + query \
        + "&display=" + str(display) \
        + "&start=" + str(start_index)
    
    request = Request(url)
    request.add_header("X-Naver-Client-Id",client_id)
    request.add_header("X-Naver-Client-Secret",client_secret)
    # request 요청에 대한 결과를 response에 저장
    response = urlopen(request)
    rescode = response.getcode()
    if(rescode==200):
        response_body = response.read()
        response_dict = json.loads(response_body.decode('utf-8'))
        items = response_dict['items']
        for item_index in range(0,len(items)):
            remove_tag = re.compile('<.*?>')
            title = re.sub(remove_tag, '', items[item_index]['title'])
            link = items[item_index]['link']
            description = re.sub(remove_tag, '',items[item_index]['description'])
            db.naver_web_api.insert_one({'title':title, 'link':link, 'description':description})
            web_df.loc[idx] = [title, link, description]
            idx += 1
            
    else:
        print("Error Code:" + rescode)

검색 질의: 낮은제주


In [17]:
result = db.naver_web_api.find()
for i in result[:10]:
    print(i)

{'_id': ObjectId('613816b37031a814457bb829'), 'title': '낮은제주', 'link': 'http://lowjeju.com/', 'description': '삼양, 파도 그리고 낮은집'}
{'_id': ObjectId('613816b37031a814457bb82a'), 'title': '낮은제주 - 홈 | Facebook', 'link': 'https://www.facebook.com/%EB%82%AE%EC%9D%80%EC%A0%9C%EC%A3%BC-423556868427546', 'description': '낮은제주, 제주도 제주. 좋아하는 사람 147명 · 16명이 방문했습니다. 일상과는 다른, 낮은 시선의 여행'}
{'_id': ObjectId('613816b37031a814457bb82b'), 'title': 'STAYFOLIO PICK. 낮은제주 - 스테이폴리오', 'link': 'https://www.stayfolio.com/picks/rym', 'description': ' STAY FOLIO 어디로 떠날까요? 언제 떠날까요? FIND STAY PROMOTION JOURNAL PRE-ORDER LOGIN STAY FOLIO 상호 : (주)스테이폴리오대표자 : 이상묵 서울특별시 종로구 자하문로9길 24, 2층(통인동) TEL : ****-*************@*********.*** 사업자등록번호:'}
{'_id': ObjectId('613816b37031a814457bb82c'), 'title': '낮은제주(@lowjeju) • Instagram 사진 및 동영상', 'link': 'https://www.instagram.com/lowjeju/', 'description': '팔로워 2,584명, 팔로잉 0명, 게시물 132개 - 낮은제주(@lowjeju)님의 Instagram 사진 및 동영상 보기'}
{'_id': ObjectId('613816b37031a814457bb82d'), 'title': '낮은 

In [24]:
# db.naver_web_api.remove()
db['naver_web_api'].drop()

In [25]:
web_df

,Title,Link,Description
0,낮은제주,http://lowjeju.com/,"삼양, 파도 그리고 낮은집"
1,낮은제주 - 홈 | Facebook,https://www.facebook.com/%EB%82%AE%EC%9D%80%EC...,"낮은제주, 제주도 제주. 좋아하는 사람 147명 · 16명이 방문했습니다. 일상과는..."
2,STAYFOLIO PICK. 낮은제주 - 스테이폴리오,https://www.stayfolio.com/picks/rym,STAY FOLIO 어디로 떠날까요? 언제 떠날까요? FIND STAY PROMO...
3,낮은제주(@lowjeju) • Instagram 사진 및 동영상,https://www.instagram.com/lowjeju/,"팔로워 2,584명, 팔로잉 0명, 게시물 132개 - 낮은제주(@lowjeju)님..."
4,"낮은 시선으로 채워진 제주의 옛집 : 낮은제주 - 글이 작품이 되는 공간, 브런치",https://brunch.co.kr/@stayfolio/245,낮은 시선으로 아늑하게 꾸며진 곳 낮은제주 제주 공항에서 동쪽으로 30분 남짓. 하...
...,...,...,...
87,제주시 시설배 가격 낮게 형성 - 미디어제주,http://www.mediajeju.com/news/articleView.html...,또 최근에 형성된 대과의 가격은 7.5kg상자당 3~4만원으로 높게 형성되나 소과의...
88,[수시] 우리품종 ‘아람콩’ 전 과정 기계화 재배 모델 정립 - 제주특별자치도청,http://www.jeju.go.kr/news/bodo/list.htm?act=v...,다운로드 아람콩 ■ 제주특별자치도 농업기술원 제주농업기술센터(소장 신양수)는 우리품...
89,2018~2022년 제주 일자리정책 5년 로드맵 - 제주특별자치도 - 제주특별자치도청,http://news.jeju.go.kr/works/jsupport/notice.h...,"이에, 낮은 일자리 질 지표를 개선하는 등 제주 실정에 맞는 일자리 정책을 마련하기..."
90,"제주은행, 3분기 '부실채권비율' 0.56%...6개 지방은행중 2번째 낮아 &lt...",http://www.jejumaeil.net/news/articleView.html...,"지난 3분기 제주은행의 대출은 전분기에 비해 증가했지만, 부실채권비율(고정이하여신비..."
